# Shiny App: Song Suggester, part I: Spotify's API Calls.
***
## About the project

This notebook demonstrates the process of making API calls to Spotify using Python. Initially, it gets an access token and then retrieves an artist's ID and name, which are provided directly by the app user. Following this, the notebook fetches a list of artists related to the input artist and their top tracks. Once this data is collected, a main() function is called to process and organize this information into a dataframe. This dataframe is then used in R for visualization in a Shiny app.


First, we import the necessary libraries:


In [1]:
import requests                      #sends request to API
import pandas as pd                  #manipulates data 
from dotenv import load_dotenv       #storing sensitive information in environment variables

import base64                        #encoding credentials for API authentification
import os                            #retrieving sensitive information from environment variables

The .env file with the access credentials is retrieved:  
* Note that this step is taken because a .env file was previously created to store sensitive information. In this case, my own *client_id* and *client_secret*.

In [3]:
load_dotenv(".env")

True

This way we can get the acces tokens for the API through a Python function:

* *client_id* and *client_secret* need to be obtained directly from the [Spotify developer site](https://developer.spotify.com/documentation/web-api).

In [4]:
def get_spotify_access_token():

    client_id = os.getenv("SPOTIFY_CLIENT_ID") 
    client_secret = os.getenv("SPOTIFY_CLIENT_SECRET")      
    auth_url = "https://accounts.spotify.com/api/token"
    credentials_b64 = base64.b64encode(f"{client_id}:{client_secret}".encode()).decode()

    headers = {"Authorization": f"Basic {credentials_b64}"}
    payload = {"grant_type": "client_credentials"}

    response = requests.post(auth_url, headers=headers, data=payload)
    response.raise_for_status()
    access_token = response.json()['access_token']
    return access_token

Functions for retrieving the artist ID, artist name, and its related artists :

In [5]:
def get_artist_id(artist_name, access_token):

    search_url = "https://api.spotify.com/v1/search"
    headers = {"Authorization": f"Bearer {access_token}"}
    params = {"q": artist_name, "type": "artist", "limit": 1}
    response = requests.get(search_url, headers=headers, params=params)
    response.raise_for_status()
    data = response.json()
    return data['artists']['items'][0]['id'] if data['artists']['items'] else None

def get_related_artists(artist_id, access_token):

    related_artists_url = f"https://api.spotify.com/v1/artists/{artist_id}/related-artists"
    headers = {"Authorization": f"Bearer {access_token}"}
    response = requests.get(related_artists_url, headers=headers)
    response.raise_for_status()
    data = response.json()
    return data['artists']

The function to get the suggested artists' top tracks needs to change a bit. In this case, we would need to iterate through the *'tracks'* list to access some features of the songs like its names, popularity indexes and its URLs. This is done so that we can identify each song by name, sort them by popularity and provide a direct link to each song.

In [6]:
def get_top_tracks_for_artists(artist_ids, access_token, market):
    top_tracks = []

    for artist_id in artist_ids:
        top_tracks_url = f"https://api.spotify.com/v1/artists/{artist_id}/top-tracks?market={market}"
        headers = {"Authorization": f"Bearer {access_token}"}
        response = requests.get(top_tracks_url, headers=headers)
        response.raise_for_status()
        data = response.json()

        
        for track in data['tracks']:
            artist_name = data["tracks"][0]["artists"][0]["name"]
            top_tracks.append({
                "artist_name": artist_name,
                "track_name": track['name'],
                "popularity": track['popularity'],
                "track_url": track['external_urls']['spotify']
            })
    return top_tracks

Finally, the functions above are called to make the calls to the API, retrieve the data, and store it into two dataframes: *top_tracks_df* and *related_artists_df*.

In [7]:
def main(artist_name, market):
    access_token = get_spotify_access_token()
    artist_id = get_artist_id(artist_name, access_token)
    if artist_id:
        related_artists = get_related_artists(artist_id, access_token)
        artist_ids = [artist['id'] for artist in related_artists]
        top_tracks = get_top_tracks_for_artists(artist_ids, access_token, market)

        top_tracks_df = pd.DataFrame(top_tracks)
        related_artists_df = pd.DataFrame(related_artists)

        return top_tracks_df, related_artists_df

    else:
        return pd.DataFrame(), pd.DataFrame()

This is what the raw *top_tracks* dataframe would look like:

In [8]:
top_tracks_df, related_artists_df = main("Black Sabbath", "US")

top_tracks_df.head(10)

,artist_name,track_name,popularity,track_url
0,Ozzy Osbourne,Crazy Train,81,https://open.spotify.com/track/7ACxUo21jtTHzy7...
1,Ozzy Osbourne,No More Tears,76,https://open.spotify.com/track/7w6PJe5KBPyvuRY...
2,Ozzy Osbourne,"Mama, I'm Coming Home",70,https://open.spotify.com/track/0S3gpZzlT9Hb7CC...
3,Ozzy Osbourne,I Don't Wanna Stop,69,https://open.spotify.com/track/5axOkQnmQmwtjr4...
4,Ozzy Osbourne,Bark at the Moon,67,https://open.spotify.com/track/2E7W1X4maFFcjHr...
5,Ozzy Osbourne,Mr. Crowley,67,https://open.spotify.com/track/2ov8L95QD25TLpZ...
6,Ozzy Osbourne,Shot in the Dark,66,https://open.spotify.com/track/31dqpLUModJWNbx...
7,Ozzy Osbourne,Dreamer,65,https://open.spotify.com/track/78PKCefXwDLbl4F...
8,Ozzy Osbourne,Close My Eyes Forever,63,https://open.spotify.com/track/43thLwzSasUUZEY...
9,Ozzy Osbourne,Hellraiser,63,https://open.spotify.com/track/4nvYDy427VvA1Tc...


Now, *related_artists*:

In [13]:
related_artists_df.head(4)

,external_urls,followers,genres,href,id,images,name,popularity,type,uri
0,{'spotify': 'https://open.spotify.com/artist/6...,"{'href': None, 'total': 5242504}","[album rock, alternative metal, birmingham met...",https://api.spotify.com/v1/artists/6ZLTlhejhnd...,6ZLTlhejhndI4Rh53vYhrY,"[{'height': 640, 'url': 'https://i.scdn.co/ima...",Ozzy Osbourne,71,artist,spotify:artist:6ZLTlhejhndI4Rh53vYhrY
1,{'spotify': 'https://open.spotify.com/artist/2...,"{'href': None, 'total': 3929366}","[album rock, alternative metal, birmingham met...",https://api.spotify.com/v1/artists/2tRsMl4eGxw...,2tRsMl4eGxwoNabM08Dm4I,"[{'height': 640, 'url': 'https://i.scdn.co/ima...",Judas Priest,66,artist,spotify:artist:2tRsMl4eGxwoNabM08Dm4I
2,{'spotify': 'https://open.spotify.com/artist/1...,"{'href': None, 'total': 3891868}","[alternative metal, hard rock, metal, rock, sp...",https://api.spotify.com/v1/artists/1DFr97A9Hnb...,1DFr97A9HnbV3SKTJFu62M,"[{'height': 640, 'url': 'https://i.scdn.co/ima...",Motörhead,64,artist,spotify:artist:1DFr97A9HnbV3SKTJFu62M
3,{'spotify': 'https://open.spotify.com/artist/4...,"{'href': None, 'total': 2520558}","[album rock, alternative metal, glam metal, ha...",https://api.spotify.com/v1/artists/4CYeVo5iZbt...,4CYeVo5iZbtYGBN4Isc3n6,"[{'height': 640, 'url': 'https://i.scdn.co/ima...",Dio,60,artist,spotify:artist:4CYeVo5iZbtYGBN4Isc3n6


## Next Steps

Some data cleaning and wrangling will be necessary to show more diversity in terms of the artists we get recommended. This can be tackled in the R part of the code.

This project continues on [Shiny App: Song Suggester, part II: Building the app in R](https://github.com/zefrios/RShiny/blob/ec5b4dd09eafbb679ff45a8c5b513464d564556b/Spotify_ArtistSuggestionsApp/Notebook_ShinyApp.ipynb)

  


***
